In [1]:
!nvidia-smi

Tue Sep 12 14:57:18 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A40                     Off | 00000000:AF:00.0 Off |                    0 |
|  0%   31C    P0              69W / 300W |   7490MiB / 46068MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, StoppingCriteria, StoppingCriteriaList
from langchain import PromptTemplate
from langchain.llms import HuggingFacePipeline
from langchain.chains import LLMChain
from langchain.embeddings import HuggingFaceInstructEmbeddings
from langchain.vectorstores import Chroma

2023-09-12 14:57:22.068530: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-12 14:57:22.933477: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
instructor_embeddings = HuggingFaceInstructEmbeddings(model_name="/root/.cache/torch/sentence_transformers/GanymedeNil_text2vec-large-chinese",
                                                      # local_files_only=True,
                                                      model_kwargs={"device": "cuda"})
persist_directory = 'db_cn'

vectordb = Chroma(persist_directory=persist_directory,
                  embedding_function=instructor_embeddings)

No sentence-transformers model found with name /root/.cache/torch/sentence_transformers/GanymedeNil_text2vec-large-chinese. Creating a new one with MEAN pooling.


In [4]:
retriever = vectordb.as_retriever(search_kwargs={"k": 3})

In [5]:
instructor_embeddings_en = HuggingFaceInstructEmbeddings(model_name="hkunlp/instructor-xl",
                                                      model_kwargs={"device": "cuda"})

persist_directory_en = 'db'

vectordb_en = Chroma(persist_directory=persist_directory_en,
                  embedding_function=instructor_embeddings_en)

load INSTRUCTOR_Transformer
max_seq_length  512


In [6]:
retriever_en = vectordb_en.as_retriever(search_kwargs={"k": 3})

## ENG LLM

In [7]:
## ENG
model_name_en = "meta-llama/Llama-2-7b-chat-hf"

tokenizer_en = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=model_name_en,
                                             local_files_only=True,
                                             # use_fast=False
                                          # use_auth_token=True,
                                         )


In [8]:
model_en = AutoModelForCausalLM.from_pretrained(model_name_en,
                                                local_files_only=True,
                                                device_map='auto',
                                                torch_dtype=torch.float16,
                                                temperature=0.2, # must be strictly positive float
                                                do_sample=True,
                                                # use_auth_token=True,
                                                #  load_in_8bit=True,
                                                #  load_in_4bit=True
                                               )
pipe_en = pipeline("text-generation",
            model=model_en,
            tokenizer= tokenizer_en,
            # return_full_text=True,
            torch_dtype=torch.bfloat16,
            device_map="auto",
            max_new_tokens = 512,
            do_sample=True,
            top_k=30,
            num_return_sequences=1,
            eos_token_id=tokenizer_en.eos_token_id
            )
llm_en = HuggingFacePipeline(pipeline=pipe_en)

template_en = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}
Helpful Answer:"""


prompt_template_en = PromptTemplate.from_template(
    template_en
)




Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

## 中文 LLM

In [9]:


model_name = "FlagAlpha/Llama2-Chinese-7b-Chat"

tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=model_name,
                                          local_files_only=True,
                                          use_fast=False
                                          # use_auth_token=True,
                                         )
model = AutoModelForCausalLM.from_pretrained(model_name,
                                         local_files_only=True,
                                         device_map='auto',
                                         torch_dtype=torch.float16,
                                         temperature=0.2, # must be strictly positive float
                                         do_sample=True,
                                         # use_auth_token=True,
                                        #  load_in_8bit=True,
                                        #  load_in_4bit=True
                                         )
pipe = pipeline("text-generation",
            model=model,
            tokenizer= tokenizer,
            # return_full_text=True,
            torch_dtype=torch.bfloat16,
            device_map="auto",
            max_new_tokens = 512,
            do_sample=True,
            top_k=30,
            num_return_sequences=1,
            eos_token_id=tokenizer.eos_token_id
            )
llm = HuggingFacePipeline(pipeline=pipe)






Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


In [10]:
template = """使用以下上下文来回答最后的问题。如果你不知道答案，就说你不知道，不要试图编造答案。

  {context}

  问题: {question}
  答案:"""


prompt_template = PromptTemplate(
    template=template, input_variables=["context", "question"]
)

chain_type_kwargs = {
    "prompt": prompt_template
}

In [11]:
from langchain.chains import RetrievalQA, RetrievalQAWithSourcesChain
qa = RetrievalQA.from_chain_type(llm=llm,
                                 chain_type="stuff",
                                 retriever=retriever,
                                 return_source_documents=True,
                                 chain_type_kwargs=chain_type_kwargs
                                )

In [12]:
question1 = "如何设定判定逻辑?"

In [18]:
%%time
res1 = qa({"query": question1})

/usr/local/lib/python3.10/site-packages/transformers/generation/utils.py:1411: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(


CPU times: user 3.44 s, sys: 354 ms, total: 3.8 s
Wall time: 3.79 s


In [20]:
res1

{'query': '如何设定判定逻辑?',
 'result': ' 设定PPID/SN的逻辑规则，用于解析保修起点日期等信息。可以在厂家保修管理后台->基础设定->判定逻辑中添加/编辑判定逻辑。\n',
 'source_documents': [Document(page_content='选择是否设定附加规则 , 限制 SN长度或厂家代码范围等 , 设定完毕后 , 点击【保存】 , \n完成该产品序列号判定逻辑设定 .', metadata={'source': 'chinese_pdf/保修管理-廠家sop-repair(去圖標).pdf', 'page': 7}),
  Document(page_content='2.2 检验逻辑设定   \n逻辑判定设置完成后 , 厂家保修管理 后台->基础设定 ->判定逻辑 , 点击进入逻辑判定\n页面, 最下方输入 PPID, 点击【解析】检验逻辑设定 ;如下图, 可依规则自动解析出厂\n时间等信息 . \n \n3. 保修类别  \n厂家保修管理 后台->基础设定 ->保修类别，点击【新增设定】 , 设定需要为产品配置\n的保修类别 ,填写服务类型、时效值及单位 , 点击【保存】完成 ; \n \n \n4. 保修约定  \n当产品保修及保修卡无法满足保修需求时 , 可添加保修约定 .', metadata={'source': 'chinese_pdf/保修管理-廠家sop-repair(去圖標).pdf', 'page': 8}),
  Document(page_content='基础设定  \n1. 判定命名  \n厂家保修管理 后台->基础设定 ->判定命名 ,依用户习惯 , 自主设定相关栏位名称 , 设定\n后, 系统相关栏位均以设定名称显示 . \n \n2. 判定逻辑  \n设定PPID/SN的逻辑规则 , 用于解析保修起点日期等信息 . \n2.1 添加/编辑判定逻辑  \n厂家保修管理 后台->基础设定 ->判定逻辑 , 点击【添加】按钮 , 添加判定逻辑 ;选定判\n定逻辑, 点击对该判定逻辑进行编辑 ; \n \n填写解码规则名称、说明、 SN样例；点击【编辑】按钮 进行应用 ; \n \n填写 PPID/SN子串区段 , 依照子串区段 的代表意义 , 可自定义规则  \n如下图中点击【

In [13]:
qa_en = RetrievalQA.from_chain_type(llm=llm_en,
                                 chain_type="stuff",
                                 retriever=retriever_en,
                                 return_source_documents=True,
                                 # chain_type_kwargs=chain_type_kwargs
                                )

In [19]:
question1 = "How can I navigate to the RMA request page in the Wareconn Customer Portal?"

In [22]:
%%time
qa_en({"query": question1})

CPU times: user 5.46 s, sys: 685 ms, total: 6.14 s
Wall time: 6.15 s


{'query': 'How can I navigate to the RMA request page in the Wareconn Customer Portal?',
 'result': ' You can navigate to the RMA request page in the Wareconn Customer Portal by following the steps outlined in the document. The steps are as follows:\n\n1. Log in to your account on the Wareconn Customer Portal by entering your email and password.\n2. Click on the "RMA Request" button in the left menu.\n3. Click on the "Add" button to create a new RMA request.\n4. Fill in the required information, including the part number (PN) and serial number (SN), and click "Submit".\nAlternatively, you can also search for the RMA request page by using the search function on the Wareconn Customer Portal.',
 'source_documents': [Document(page_content='2 / 6 \nThis document  belongs to Wareconn  Technology Services (Tianjin) Co., Ltd. It is only intended to be \nused to introduce Wareconn functions and procedures. Please do not use it for other purposes.  2 wareconn standard operating procedure  \nSOP 

In [22]:
!nvidia-smi

Tue Sep 12 14:30:30 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A40                     Off | 00000000:AF:00.0 Off |                    0 |
|  0%   30C    P0              69W / 300W |  33683MiB / 46068MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [14]:
from fastapi import FastAPI
from pydantic import BaseModel, Field
from typing import Optional, List
app = FastAPI()
class Prompt(BaseModel):
    # context: str
    question: str
    lang: str
    model: str = Field(default="FlagAlpha/Llama2-Chinese-7b-Chat")
    temperature: float = Field(default=0.2)

@app.post("/question_answering")
async def get_completion(input: Prompt):
    input_dict = input.dict()

    if input_dict["lang"] == "en_us":
        
        ans = qa_en({"query": input_dict["question"]})
    else:
      
        ans = qa({"query": input_dict["question"]})
    return ans
@app.get("/")
async def root():
    return {
        "message": "this is get"
    }

In [15]:
class Documents(BaseModel):
    input: List[str]
    lang: str
@app.post("/embeddings")
async def embed_docs(input: Documents):
    input_dict = input.dict()
    docs = input_dict["input"]
    
    if input_dict["lang"] == "en_us":
        embed_docs = instructor_embeddings_en.embed_documents(docs)
    else:
        embed_docs = instructor_embeddings.embed_documents(docs)

    data = [{"embedding": emb, "index": idx} for idx, emb in enumerate(embed_docs)]
    return {
        "data": data,
    }

In [16]:
class Question(BaseModel):
    user_input:str
    lang: str
    k: Optional[int] = Field(default=3)


@app.post("/get_relevant_docs")
async def retrieve_docs(input_q: Question):
    input_dict = input_q.dict()

    if input_dict["lang"] == "en_us":
        ans = retriever_en.get_relevant_documents(input_dict["user_input"])
    else:
        ans = retriever.get_relevant_documents(input_dict["user_input"])

    docs = list(map(lambda a: {
            'page_content': a.page_content,
            'source': a.metadata["source"],
            'page': a.metadata["page"]}, ans))
    return {
        # "user_input": question["user_input"],
        "relavant_docs": docs
    }

In [17]:
# %load_ext autoreload
# %autoreload 2

In [18]:
import nest_asyncio

# Allow for asyncio to work within the Jupyter notebook cell
nest_asyncio.apply()

In [19]:
import uvicorn
uvicorn.run(app=app, host="127.0.0.1", port=8006)

INFO:     Started server process [31655]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8006 (Press CTRL+C to quit)


INFO:     127.0.0.1:5169 - "POST /get_relevant_docs HTTP/1.1" 200 OK
INFO:     127.0.0.1:5171 - "POST /get_relevant_docs HTTP/1.1" 200 OK
INFO:     127.0.0.1:5183 - "POST /embeddings HTTP/1.1" 200 OK
INFO:     127.0.0.1:5185 - "POST /embeddings HTTP/1.1" 200 OK
INFO:     127.0.0.1:5187 - "POST /embeddings HTTP/1.1" 200 OK


error uploading: HTTPSConnectionPool(host='app.posthog.com', port=443): Max retries exceeded with url: /batch/ (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7fe079d3ea70>: Failed to establish a new connection: [Errno -2] Name or service not known'))
INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [31655]


In [6]:
!pip show fastapi

Name: fastapi
Version: 0.100.0
Summary: FastAPI framework, high performance, easy to learn, fast to code, ready for production
Home-page: 
Author: 
Author-email: Sebastián Ramírez <tiangolo@gmail.com>
License: 
Location: /usr/local/lib/python3.10/site-packages
Requires: pydantic, starlette, typing-extensions
Required-by: chromadb
